In [ ]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import pickle

In [ ]:
#Daten einlesen
data = pd.read_csv(r'C:\Beispielnutzer\sdm_meldungen_processed_2021_2022.csv', sep=",", engine="python")

In [ ]:
# Funktion für Schwellenwert
def dropLowFrequency(df : object, y_col : str, th : int):
    dfdropped = data.groupby(y_col).filter(lambda x : (x[y_col].count()>th).any())
    return dfdropped

In [ ]:
# Anwenden der Schwellenwertsfunktion
dfdropped = dropLowFrequency(data,'gruppe_id', 10)

In [ ]:
# Abhängige Variable (y) und unabhängige Variable (x) definieren
y = dfdropped['gruppe_id']
X = dfdropped['input']

In [ ]:
# Trainings- Testmenge
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=42, stratify=dfdropped['gruppe_id'])

In [ ]:
# Label encoding
Encoder = LabelEncoder()
y_train_fit = Encoder.fit_transform(y_train)
y_test_fit = Encoder.fit_transform(y_test)

In [ ]:
# Embedding
Tfidf_vect = TfidfVectorizer()
X_train_Tfidf = Tfidf_vect.fit_transform(X_train.values.astype(str))
X_test_Tfidf = Tfidf_vect.transform(X_test)

In [ ]:
#Alternative 1
# Logistische Regressionsmodell
lr_tfidf=LogisticRegression(solver = 'sag', C=10, penalty = 'l2')

lr_tfidf.fit(X_train_Tfidf, y_train)

# Vorhersagen
y_predict = lr_tfidf.predict(X_test_Tfidf)

# Klassifikationsreport
print(classification_report(y_test,y_predict))

In [ ]:
# Alternative 2
"""
# SVM Modell vorbereiten
svc = svm.LinearSVC()

#Modell mithilfe des Trainingsdatensatzes trainieren (X-train_Tfidf als unabhängige Variable und y_train als abhängige Variable)
svc.fit(X_train_Tfidf,y_train_fit)

# Lable des Testdatensatzes vorhersagen und in predictions_SVC speichern
predictions_SVC = svc.predict(X_test_Tfidf)
"""

In [ ]:
#Modell speichern
filename = 'model.pkl'
pickle.dump(lr_tfidf, open(filename, 'wb'))